# Limit Opinion Consensus Check for Symmetric Case

#### Unified Model with Multiple Strategic Agents
#### Strategic Agent-Specific Message Weights

James Yu, 20 January 2023

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def M(K, B, R, L, delta):
    """Computes M_{t-1} given B_l \forall l, K_t^l \forall l, 
        R_l \forall l, number of strategic agents L, and delta."""
    # handle the generic structure first, with the correct pairings:
    base = [[B[l_prime].T @ K[l_prime] @ B[l] for l in range(L)] for l_prime in range(L)]
    # then change the diagonals to construct M_{t-1}:
    for l in range(L): base[l][l] = B[l].T @ K[l] @ B[l] + R[l]/delta
    return np.block(base)

def H(B, K, A, L):
    """Computes H_{t-1} given B_l \forall l, K_t^l \forall l, 
        A, and number of strategic agents L."""
    return np.concatenate(tuple(B[l].T @ K[l] @ A for l in range(L)), axis = 0)

def C_l(A, B, K, k, h, L, c, x, n):
    """Computes C_{t-1}^h (displayed as C_{t-1}^l) given A, B_l \forall l, K_t^l \forall l, 
        k_t^l \forall l, a specific naive agent h, number of strategic agents L, 
        c_l \forall l, x_l \forall l, and number of naive agents n"""
    return np.concatenate(tuple(B[l].T @ K[l] @ A @ ((x[h] - x[l]) * np.ones((n, 1))) 
                           + B[l].T @ K[l] @ c[l] 
                           + 0.5 * B[l].T @ k[l].T for l in range(L)), axis = 0)

def E(M_, H_):
    """Computes the generic E_{t-1} given M_{t-1} and H_{t-1}."""
    return np.linalg.inv(M_) @ H_

def F(M_, C_l_, l, n):
    """Computes F_{t-1}^l given M_{t-1}, C_{t-1}^l, 
       specific naive agent l and number of naive agents n."""
    return (np.linalg.inv(M_) @ C_l_)[l*n:(l+1)*n, :] # e.g. l = 0 gives ln = 0, l = 1 gives ln = n, etc

def G(A, B, E_, L, n):
    """Computes the generic G_{t-1} given A, B_l \forall l, 
        E_{t-1}, number of strategic agents L, and number of naive agents n."""
    return A - sum([B[l] @ E_[l*n:(l+1)*n, :] for l in range(L)])
    
def g_l(B, E_, h, x, F_, L, n, c):
    """Computes g_{t-1}^l given B_l \forall l, E_{t-1}^l, 
        a particular naive agent h, x_l \forall l, F_{t-1}^l \forall l, 
        number of strategic agents L, number of naive agents n, and c_h."""
    return - sum([B[l] @ (E_[l*n:(l+1)*n, :] @ ((x[h] - x[l]) * np.ones((n, 1))) + F_[l]) for l in range(L)]) + c[h]

In [3]:
def K_t_minus_1(Q, K, E_, R, G_, L, delta, n):
    return [Q[l] + E_[l*n:(l+1)*n, :].T @ R[l] @ E_[l*n:(l+1)*n, :] 
            + delta * G_.T @ K[l] @ G_ for l in range(L)]

def k_t_minus_1(K, k, G_, g, E_, F_, R, L, delta, n):
    return [2*delta* g[l].T @ K[l] @ G_ + delta * k[l] @ G_ 
            + 2 * F_[l].T @ R[l] @ E_[l*n:(l+1)*n, :] for l in range(L)]

def kappa_t_minus_1(K, k, kappa, g_, F_, R, L, delta):            
    return [-delta * (g_[l].T @ K[l] @ g_[l] + k[l] @ g_[l] - kappa[l]) 
            - (F_[l].T @ R[l] @ F_[l]) for l in range(L)]

In [4]:
def should_terminate(bundles, eps):
    return all([np.allclose(b[0], b[1], rtol = eps, atol = eps) for b in bundles])

def solve(K_t, k_t, kappa_t, A, B, delta, n, L, Q, R, x, c, tol):
    historical_K = [K_t]
    historical_k = [k_t]
    historical_kappa = [kappa_t]
    eps = np.sqrt(np.finfo(np.float64).eps)
    while True:
        M_ = M(K_t, B, R, L, delta)
        H_ = H(B, K_t, A, L)
        E_ = E(M_, H_)
        G_ = G(A, B, E_, L, n)
        K_new = K_t_minus_1(Q, K_t, E_, R, G_, L, delta, n)
        F_ = [F(M_, C_l(A, B, K_t, k_t, l, L, c, x, n), l, n) for l in range(L)]
        g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
        k_new = k_t_minus_1(K_t, k_t, G_, g, E_, F_, R, L, delta, n)
        kappa_new = kappa_t_minus_1(K_t, k_t, kappa_t, g, F_, R, L, delta)
        historical_K.insert(0, K_new)
        historical_k.insert(0, k_new)
        historical_kappa.insert(0, kappa_new)
        if should_terminate([(K_t, K_new), (k_t, k_new), (kappa_t, kappa_new)], eps):
            return historical_K[0], historical_k[0], historical_kappa[0]
        K_t = K_new
        k_t = k_new
        kappa_t = kappa_new

In [5]:
def optimal(X_init, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, eps):
    X_t = [a.copy() for a in X_init]
    xs = defaultdict(list)
    for l in range(L):
        xs[l].append(X_t[l])
        
    rs = defaultdict(list)
    payoffs = defaultdict(list)
    payoff = defaultdict(int)
    
    M_ = M(K_ss, B, R, L, delta)
    H_ = H(B, K_ss, A, L)
    E_ = E(M_, H_)
    G_ = G(A, B, E_, L, n)
    F_ = [F(M_, C_l(A, B, K_ss, k_ss, l, L, c, x, n), l, n) for l in range(L)]
    g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
    
    i = 0
    while True:
        for l in range(L):
            # TODO: special code for finite horizon needs a terminal time T term
            Y_new = -1 * E_[l*n:(l+1)*n, :] @ X_t[l] - F(M_, C_l(A, B, K_ss, k_ss, l, L, c, x, n), l, n)
            rs[l].append(Y_new)
            payoff[l] += (-1 * delta**i * (X_t[l].T @ Q[l] @ X_t[l])).item() + (-1 * delta**i * (Y_new.T @ R[l] @ Y_new)).item()
            payoffs[l].append(payoff[l])
            X_new = G_ @ X_t[l] + g[l]
            xs[l].append(X_new)
            if l == L - 1 and np.allclose(X_t[l], X_new, rtol = eps, atol = eps):
                return xs, rs, payoffs
            X_t[l] = X_new 
        i += 1
        
    return xs, rs, payoffs

In [6]:
def run_simulation(b1, b2, a1, a2, delta = 0.9, c = 200, X_0_1 = np.array([[10.0, -5.0, 5.0]], ndmin = 2).T, symmetric = True, print_ = True, tol = np.sqrt(np.finfo(np.float64).eps)):
    if symmetric:
        A = np.array([ # stubborn
            [0.9, 0.07, 0.03],
            [0.07, 0.58, 0.35],
            [0.03, 0.35, 0.62]
        ])
        if print_: print("SYMMETRIC NETWORK")
    else:
        raise Exception("asymmetric network not allowed")
    X_0 = [X_0_1 - b1, X_0_1 - b2]
    n = 3 # number of naive agents
    L = 2 # number of strategic agents
    Q = [np.identity(n), np.identity(n)] # objective function for messages is just X'IX = X'X
    R = [c * np.identity(n), c * np.identity(n)] # message cost R = cI_n for some c under the new notation
    B = [a1 * np.identity(n), a2 * np.identity(n)] # B^l = a_l I_n
    x = [b1, b2] # agendas
    r = [0, 0] # message cost minimality is centered around zero
    c_base = sum([B[l] @ (r[l] * np.ones((n, 1))) for l in range(L)])
    c = [c_base + (A - np.identity(n)) @ (x[l] * np.ones((n, 1))) for l in range(L)] # normalization vector
    
    K_ss, k_ss, kappa_ss = solve(Q, [np.zeros((1, n)), np.zeros((1, n))], [0, 0], A, B, delta, n, L, Q, R, x, c, tol)
    xs, rs, payoffs = optimal(X_0, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, tol)
    if print_:
        print("Steady-State Opinions:")
        print(xs[0][-1] + b1)
        print("Average of agendas:")
        print((b1+b2)/2)
        print()
        print("K^* (1, 2):")
        print(K_ss[0])
        print(K_ss[1])
        print()
        print("k^* (1, 2):")
        print(k_ss[0])
        print(k_ss[1])
        print()
        print("kappa^* (1, 2):")
        print(kappa_ss[0])
        print(kappa_ss[1])
    if print_ == False:
        return xs[0][-1] + b1

In [7]:
run_simulation(0, 0, 1, 1) #b1, b2 agendas, followed by alpha_1, alpha_2 weights

SYMMETRIC NETWORK
Steady-State Opinions:
[[2.7302074e-07]
 [2.7302074e-07]
 [2.7302074e-07]]
Average of agendas:
0.0

K^* (1, 2):
[[3.78777258 1.1435288  1.03448857]
 [1.1435288  2.91545075 1.9068104 ]
 [1.03448857 1.9068104  3.02449098]]
[[3.78777258 1.1435288  1.03448857]
 [1.1435288  2.91545075 1.9068104 ]
 [1.03448857 1.9068104  3.02449098]]

k^* (1, 2):
[[0. 0. 0.]]
[[0. 0. 0.]]

kappa^* (1, 2):
[[-0.]]
[[-0.]]


In [8]:
run_simulation(10000, 1000000, 5000, 600)

SYMMETRIC NETWORK
Steady-State Opinions:
[[11625.87638798]
 [11625.87638726]
 [11625.87638774]]
Average of agendas:
505000.0

K^* (1, 2):
[[1.00000634e+00 8.87061121e-07 5.44986774e-07]
 [8.87061121e-07 1.00000361e+00 3.28158155e-06]
 [5.44986774e-07 3.28158155e-06 1.00000395e+00]]
[[1.00000009e+00 1.27811501e-08 7.85234212e-09]
 [1.27811501e-08 1.00000005e+00 4.72828058e-08]
 [7.85234212e-09 4.72828058e-08 1.00000006e+00]]

k^* (1, 2):
[[25581.99915085 25581.99915086 25581.99915086]]
[[-25597.17070242 -25597.1707023  -25597.17070168]]

kappa^* (1, 2):
[[-6.3133415e+14]]
[[-4.38689529e+16]]


In [9]:
run_simulation(0, 5, 0.5, 0.6, c = 0.001)

SYMMETRIC NETWORK
Steady-State Opinions:
[[3.29858496]
 [3.29858496]
 [3.29858496]]
Average of agendas:
2.5

K^* (1, 2):
[[1.00054838e+00 7.66793942e-05 4.71020401e-05]
 [7.66793942e-05 1.00031176e+00 2.83720873e-04]
 [4.71020401e-05 2.83720873e-04 1.00034134e+00]]
[[1.00078899e+00 1.10369149e-04 6.78155985e-05]
 [1.10369149e-04 1.00044856e+00 4.08244004e-04]
 [6.78155985e-05 4.08244004e-04 1.00049112e+00]]

k^* (1, 2):
[[3.86599532 3.86599532 3.86599532]]
[[-3.86304519 -3.86304519 -3.86304519]]

kappa^* (1, 2):
[[-166665.88961279]]
[[-115621.20489165]]


In [10]:
run_simulation(0, 50000000, 0.5, 0.6, c = 0.001)

SYMMETRIC NETWORK
Steady-State Opinions:
[[32985849.61916232]
 [32985849.61915815]
 [32985849.61915527]]
Average of agendas:
25000000.0

K^* (1, 2):
[[1.00054838e+00 7.66793942e-05 4.71020401e-05]
 [7.66793942e-05 1.00031176e+00 2.83720873e-04]
 [4.71020401e-05 2.83720873e-04 1.00034134e+00]]
[[1.00078899e+00 1.10369149e-04 6.78155985e-05]
 [1.10369149e-04 1.00044856e+00 4.08244004e-04]
 [6.78155985e-05 4.08244004e-04 1.00049112e+00]]

k^* (1, 2):
[[38659953.1800124  38659953.18001056 38659953.18001081]]
[[-38630451.8761257  -38630451.87612632 -38630451.87612429]]

kappa^* (1, 2):
[[-1.6666589e+19]]
[[-1.15621205e+19]]


In [11]:
run_simulation(0, 50000000, 0.5, 0.6)

SYMMETRIC NETWORK
Steady-State Opinions:
[[29912158.04595353]
 [29912158.04595353]
 [29912158.04595353]]
Average of agendas:
25000000.0

K^* (1, 2):
[[4.4403969  1.68498693 1.56834678]
 [1.68498693 3.5072757  2.50146797]
 [1.56834678 2.50146797 3.62391585]]
[[4.51804633 1.75568218 1.63856629]
 [1.75568218 3.58111925 2.57549338]
 [1.63856629 2.57549338 3.69823513]]

k^* (1, 2):
[[77263792.38263299 77263792.38263297 77263792.38263297]]
[[-55407371.95950853 -55407371.95950856 -55407371.95950855]]

kappa^* (1, 2):
[[-1.45131421e+15]]
[[-7.11990399e+14]]


In [12]:
run_simulation(-20, 5, 0.5, 0.6, c = 0.001, delta = 0.9)

SYMMETRIC NETWORK
Steady-State Opinions:
[[-3.50707519]
 [-3.50707519]
 [-3.50707519]]
Average of agendas:
-7.5

K^* (1, 2):
[[1.00054838e+00 7.66793942e-05 4.71020401e-05]
 [7.66793942e-05 1.00031176e+00 2.83720873e-04]
 [4.71020401e-05 2.83720873e-04 1.00034134e+00]]
[[1.00078899e+00 1.10369149e-04 6.78155985e-05]
 [1.10369149e-04 1.00044856e+00 4.08244004e-04]
 [6.78155985e-05 4.08244004e-04 1.00049112e+00]]

k^* (1, 2):
[[19.32997659 19.32997659 19.32997659]]
[[-19.31522594 -19.31522594 -19.31522594]]

kappa^* (1, 2):
[[-4166647.2403197]]
[[-2890530.12229136]]


In [13]:
run_simulation(10, 100, 0.5, 0.50002, c = 0.001, delta = 0.9)

SYMMETRIC NETWORK
Steady-State Opinions:
[[55.00327353]
 [55.00327353]
 [55.00327353]]
Average of agendas:
55.0

K^* (1, 2):
[[1.00081573e+00 1.14090319e-04 7.00940502e-05]
 [1.14090319e-04 1.00046376e+00 4.22064204e-04]
 [7.00940502e-05 4.22064204e-04 1.00050776e+00]]
[[1.00081580e+00 1.14099434e-04 7.00996555e-05]
 [1.14099434e-04 1.00046380e+00 4.22097887e-04]
 [7.00996555e-05 4.22097887e-04 1.00050780e+00]]

k^* (1, 2):
[[73.8434663 73.8434663 73.8434663]]
[[-73.84345193 -73.84345193 -73.84345193]]

kappa^* (1, 2):
[[-40863124.55921516]]
[[-40859844.82597104]]
